In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
pitches = pd.read_csv('k_prob.csv')

#to_drop = ['game_pk','game_date','at_bat_number','pitch_number','pitch_type','pitcher_name','pitcher','batter','catcher','stand','p_throws','broadcast']
#pitches.drop(to_drop, inplace=True, axis=1)
pitches.description[pitches['description'] == 'called_strike'] = 0
pitches.description[pitches['description'] == 'ball'] = 1
pitches['description'].value_counts()
pitches.dropna(inplace=True, axis=0) # drop all non number values

C:\Users\andre\AppData\Local\Temp\ipykernel_81564\354896931.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitches.description[pitches['description'] == 'called_strike'] = 0
C:\Users\andre\AppData\Local\Temp\ipykernel_81564\354896931.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitches.description[pitches['description'] == 'ball'] = 1


In [15]:
#This is the normalisation section
new_top = pitches['sz_top'].mean()
new_bot = pitches['sz_bot'].mean()

normalised_values = []

for index, row in pitches.iterrows():
    if row['plate_z'] > row['sz_top']:
        normalised_values.append(row['plate_z'] - row['sz_top'] + new_top)
    
    elif row['plate_z'] < row['sz_bot']:
        normalised_values.append(row['plate_z'] / row['sz_bot'] * new_bot)

    else:
        normalised_values.append((row['plate_z']-row['sz_bot']) / (row['sz_top']-row['sz_bot']) * (new_top-new_bot) + new_bot)

pitches.insert(19,'Normalised_Values',normalised_values, True)
#end of section

In [16]:
p_pos = ['plate_x', 'Normalised_Values','zone', 'sz_top', 'sz_bot', 'strikes']
X = pitches[p_pos] # Features
y = pitches.description # Target variable
y = y.astype("str")


In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size= 0.10)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=15)
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89     11562
           1       0.94      0.95      0.95     23525

    accuracy                           0.93     35087
   macro avg       0.92      0.91      0.92     35087
weighted avg       0.93      0.93      0.93     35087

[[10129  1433]
 [ 1151 22374]]


In [18]:
from sklearn.model_selection import cross_val_score

print("Array of scores for the Kfold = 10 validation")
print(cross_val_score(classifier,X,y, cv = 10))

print("Average score for K = 10 KFold validation")
print(np.mean(cross_val_score(classifier,X,y, cv = 10)))

Array of scores for the Kfold = 10 validation
[0.92567048 0.92951806 0.92854904 0.93131359 0.9305991  0.93173916
 0.93048509 0.93384826 0.92903152 0.93225218]
Average score for K = 10 KFold validation
0.930300646835893


In [19]:
print(np.min(cross_val_score(classifier, X, y, cv=10)))
print(np.max(cross_val_score(classifier, X, y, cv=10)))

0.9256704762447631
0.9338482585646697
